In [1]:
from lib.word_generation import *
model = WordGeneration(hid_dim=256, num_layers=1, load_data=False)

/home/haruka/Workspace/seq2seq-summary/env/bin/python


In [7]:
model.predict('it met with positive sales in japan ,  and was praised by')

'it met with positive sales in japan ,  and was praised by album around who county games out total them run against h final set south released york u such america weeks british support may only million also united only country line university two said international back population february later days group international re side july government south during known between h all number german line football reached water side very became six then weeks population song later where weeks game a . february august before season people made single july british lead league way though games these april season british long central days , episode both use way field highway because '

In [6]:
from collections import Counter, defaultdict
import numpy as np
data_name = 'wiki'
with open(f'data/{data_name}.txt', encoding='utf-8') as f:
    text = f.read().lower()
text = list(filter(lambda x: x!='', text.split(' ')))
words = {words for words, count in Counter(text).items() if count >=80}
print(f'Have {len(words)} of words')
word_indices = defaultdict(lambda:0, {c: i for i, c in enumerate(sorted(words),1)})
indices_word = defaultdict(lambda:'', {c: i for i, c in word_indices.items()})
maxlen = 100
step = 5
sentences = []
next_word = []
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_word.append(text[i + maxlen])
x = np.zeros((len(sentences)+1, maxlen+1, len(words)+1),dtype=bool)
y = np.zeros(len(sentences)+1,dtype=bool)
for i, sentence in enumerate(sentences):
    for t, word in enumerate(sentence):
        x[i, t, word_indices[word]] = 1
    y[i] = word_indices[next_word[i]]

Have 291 of words
